## PART ONE

This Section has code for the first part of the Project

In [23]:
import requests 
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd 

In [24]:
## wikipedia URL with source Data
data_source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp = requests.get(data_source).text

In [25]:
soup = BeautifulSoup(resp, 'xml')#Beautiful Soup to Parse the url page

In [26]:
table=soup.find('table')

In [27]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [28]:
# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [29]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [34]:
# remove rows where Borough is 'Not assigned'
df=df[df['Borough']!='Not assigned']

In [35]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [39]:
# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
#df[df['Neighbourhood']=='Not assigned']=df['Borough']
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df=df.reset_index(drop=True)


In [40]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [41]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [42]:
# join the newly constructed joined data frame
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [43]:
# drop the Neighbourhood column
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [44]:
df_merge.drop_duplicates(inplace=True)

In [45]:
# rename Neighbourhood_joined back to Neighbourhood
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [49]:
df_merge.shape

(103, 3)

#End of part one